## Movie Recommendation System using Boltzmann Machine

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torch.optim as optim

In [2]:
movie = pd.read_csv('data/movies.dat', sep = '::', engine = 'python', header = None, encoding = 'latin-1')
rating = pd.read_csv('data/ratings.dat', sep = '::', engine = 'python', header = None, encoding = 'latin-1')
user = pd.read_csv('data/users.dat', sep = '::', engine = 'python', header = None, encoding = 'latin-1')

In [3]:
print ("movie data shape" + str(movie.shape))
print ("rating data shape" + str(rating.shape))
print ("user data shape" + str(user.shape))

movie data shape(3883, 3)
rating data shape(1000209, 4)
user data shape(6040, 5)


In [4]:
movie.head(5)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
rating.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
user.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
data = pd.read_csv('data-K cross validation/u1.base', delimiter='\t')
data = np.array(data, dtype='int')

In [8]:
test = pd.read_csv('data-K cross validation/u1.test', delimiter='\t')
test = np.array(test, dtype='int')

In [9]:
print (data.shape)
print (test.shape)

(79999, 4)
(19999, 4)


In [10]:
data[:10]

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712],
       [        1,         7,         4, 875071561],
       [        1,         8,         1, 875072484],
       [        1,         9,         5, 878543541],
       [        1,        11,         2, 875072262],
       [        1,        13,         5, 875071805],
       [        1,        15,         5, 875071608]])

In [11]:
test[:10]

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198],
       [        1,        20,         4, 887431883],
       [        1,        23,         4, 875072895],
       [        1,        24,         3, 875071713],
       [        1,        27,         2, 876892946],
       [        1,        31,         3, 875072144],
       [        1,        33,         4, 878542699]])

In [15]:
total_users = int(max(max(data[:,0]), max(test[:,0])))
total_movies = int(max(max(data[:,1]), max(test[:,1])))

In [17]:
print (total_users)
print (total_movies)

943
1682


In [18]:
def convert(inputs):
    new_data = []
    for users in range(1, total_users + 1):
        id_movies = inputs[:,1][inputs[:,0] == users]
        id_ratings = inputs[:,2][inputs[:,0] == users]
        ratings = np.zeros(total_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [19]:
train = convert(data)
test = convert(test)

In [22]:
train = torch.FloatTensor(train)
test = torch.FloatTensor(test)

In [24]:
train, test

(
     0     3     4  ...      0     0     0
     4     0     0  ...      0     0     0
     0     0     0  ...      0     0     0
        ...          ⋱          ...       
     5     0     0  ...      0     0     0
     0     0     0  ...      0     0     0
     0     5     0  ...      0     0     0
 [torch.FloatTensor of size 943x1682], 
     0     0     0  ...      0     0     0
     0     0     0  ...      0     0     0
     0     0     0  ...      0     0     0
        ...          ⋱          ...       
     0     0     0  ...      0     0     0
     0     0     0  ...      0     0     0
     0     0     0  ...      0     0     0
 [torch.FloatTensor of size 943x1682])

In [25]:
train[train == 0] = -1
train[train == 1] = 0
train[train == 2] = 0
train[train >= 3] = 1

In [26]:
test[test == 0] = -1
test[test == 1] = 0
test[test == 2] = 0
test[test >= 3] = 1

In [27]:
class RBM():
    def __init__(self, v, h):
        self.W = torch.randn(h, v)
        self.a = torch.randn(1, h)
        self.b = torch.randn(1, v)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [31]:
v = len(train[0])
h = 100
batch_size = 100
epochs = 100

In [32]:
rbm = RBM(v, h)

In [33]:
for epoch in range(1, epochs + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, total_users - batch_size, batch_size):
        vk = train[id_user : id_user + batch_size]
        v0 = train[id_user : id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

/anaconda/lib/python3.6/site-packages/torch/tensor.py:297: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)


epoch: 1 loss: 0.29123673850260734
epoch: 2 loss: 0.2506326410887169
epoch: 3 loss: 0.2487668384264279
epoch: 4 loss: 0.25038637160951616
epoch: 5 loss: 0.25107360096611253
epoch: 6 loss: 0.25042107904304534
epoch: 7 loss: 0.2498715987966832
epoch: 8 loss: 0.2501224798025557
epoch: 9 loss: 0.2504299396161946
epoch: 10 loss: 0.24890871853245924
epoch: 11 loss: 0.25083853115237265
epoch: 12 loss: 0.2491661924271079
epoch: 13 loss: 0.25075083673387905
epoch: 14 loss: 0.25123829353104593
epoch: 15 loss: 0.2504830468433176
epoch: 16 loss: 0.2513916187665638
epoch: 17 loss: 0.2502117104824085
epoch: 18 loss: 0.25159430768537905
epoch: 19 loss: 0.25094811289710367
epoch: 20 loss: 0.25063822190603924
epoch: 21 loss: 0.25067560771051134
epoch: 22 loss: 0.2500304084209339
epoch: 23 loss: 0.24962682769731037
epoch: 24 loss: 0.25236863409094745
epoch: 25 loss: 0.2500781738321538
epoch: 26 loss: 0.2533806398358177
epoch: 27 loss: 0.25197683420625583
epoch: 28 loss: 0.2501459712113012
epoch: 29 loss

In [34]:
test_loss = 0
s = 0.
for id_user in range(total_users):
    v = train[id_user : id_user + 1]
    vt = test[id_user : id_user + 1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.2345256809806862
